In [1]:
import json

In [13]:
import boto3
import pandas as pd

In [3]:
session = boto3.session.Session()

In [4]:
sts = session.client('sts')
sts.get_caller_identity()

{'UserId': 'AROA5TWTPH6BVIRFIP7Q6:eoin.shanaghy@fourtheorem.com',
 'Account': '935672627075',
 'Arn': 'arn:aws:sts::935672627075:assumed-role/AWSReservedSSO_DevAdministratorAccess_427a06cd5ebe9968/eoin.shanaghy@fourtheorem.com',
 'ResponseMetadata': {'RequestId': '9c9d4499-fb58-4bbf-aa4c-729ad85f0d22',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9c9d4499-fb58-4bbf-aa4c-729ad85f0d22',
   'content-type': 'text/xml',
   'content-length': '519',
   'date': 'Fri, 24 Nov 2023 14:49:48 GMT'},
  'RetryAttempts': 0}}

In [5]:
lambda_client = session.client('lambda')

In [6]:
paginator = lambda_client.get_paginator('list_functions')

In [9]:
iter = paginator.paginate()
function_names = [f['FunctionName'] for page in iter for f in page['Functions']]
measurement_function_names = [fn for fn in function_names if fn.startswith('perf_') and fn.endswith('_import_measure')]
measurement_function_names

['perf_zip_Python39_import_measure',
 'perf_image_Python39_import_measure',
 'perf_zip_layers_Python38_import_measure',
 'perf_zip_Python310_import_measure',
 'perf_zip_Python311_import_measure',
 'perf_zip_layers_Python311_import_measure',
 'perf_image_Python311_import_measure',
 'perf_zip_layers_Python310_import_measure',
 'perf_zip_layers_Python39_import_measure',
 'perf_image_Python38_import_measure',
 'perf_zip_Python38_import_measure',
 'perf_image_Python310_import_measure']

In [10]:
responses = {fn: lambda_client.invoke(FunctionName=fn, InvocationType="RequestResponse") for fn in measurement_function_names}    

In [11]:
results = {fn: json.load(val['Payload']) for fn, val in responses.items()}

In [12]:
results

{'perf_zip_Python39_import_measure': {'numpy': 834658.898,
  'pandas': 3097687.964,
  'pyarrow': 3.237,
  'pyarrow.parquet': 185615.269,
  'boto3': 183934.103},
 'perf_image_Python39_import_measure': {'numpy': 890267.64,
  'pandas': 3198348.908,
  'pyarrow': 3.567,
  'pyarrow.parquet': 188608.51,
  'boto3': 197334.155},
 'perf_zip_layers_Python38_import_measure': {'numpy': 832815.883,
  'pandas': 3124460.797,
  'pyarrow': 3.08,
  'pyarrow.parquet': 202379.219,
  'boto3': 195430.227},
 'perf_zip_Python310_import_measure': {'numpy': 862707.933,
  'pandas': 3156157.823,
  'pyarrow': 2.404,
  'pyarrow.parquet': 187799.26,
  'boto3': 194015.137},
 'perf_zip_Python311_import_measure': {'numpy': 874864.23,
  'pandas': 3250313.898,
  'pyarrow': 3.266,
  'pyarrow.parquet': 187586.673,
  'boto3': 179576.619},
 'perf_zip_layers_Python311_import_measure': {'numpy': 829189.137,
  'pandas': 3037320.064,
  'pyarrow': 2.599,
  'pyarrow.parquet': 197616.677,
  'boto3': 160672.473},
 'perf_image_Python3

In [15]:
flattened_data = []
for runtime_function, modules in results.items():
    for module, load_time in modules.items():
        flattened_data.append({'runtime_function': runtime_function, 'module': module, 'load_time': load_time})

df = pd.DataFrame(flattened_data)

pivot_df = df.pivot(index='runtime_function', columns='module', values='load_time')

print(pivot_df)

module                                         boto3       numpy       pandas  \
runtime_function                                                                
perf_image_Python310_import_measure       191124.471  851961.176  3275334.281   
perf_image_Python311_import_measure       168108.413  850410.215  3086290.574   
perf_image_Python38_import_measure        179482.829  776178.184  2895664.965   
perf_image_Python39_import_measure        197334.155  890267.640  3198348.908   
perf_zip_Python310_import_measure         194015.137  862707.933  3156157.823   
perf_zip_Python311_import_measure         179576.619  874864.230  3250313.898   
perf_zip_Python38_import_measure          195804.756  794281.014  3065766.095   
perf_zip_Python39_import_measure          183934.103  834658.898  3097687.964   
perf_zip_layers_Python310_import_measure  188822.421  878432.048  3272346.674   
perf_zip_layers_Python311_import_measure  160672.473  829189.137  3037320.064   
perf_zip_layers_Python38_imp

In [18]:
pivot_df

module,boto3,numpy,pandas,pyarrow,pyarrow.parquet,total_load_time_seconds
runtime_function,,,,,,
perf_image_Python310_import_measure,191124.471,851961.176,3275334.281,2.657,187478.219,4.505901
perf_image_Python311_import_measure,168108.413,850410.215,3086290.574,3.118,196061.877,4.300874
perf_image_Python38_import_measure,179482.829,776178.184,2895664.965,2.531,187800.165,4.039129
perf_image_Python39_import_measure,197334.155,890267.640,3198348.908,3.567,188608.510,4.474563
perf_zip_Python310_import_measure,194015.137,862707.933,3156157.823,2.404,187799.260,4.400683
perf_zip_Python311_import_measure,179576.619,874864.230,3250313.898,3.266,187586.673,4.492345
perf_zip_Python38_import_measure,195804.756,794281.014,3065766.095,2.853,195973.283,4.251828
perf_zip_Python39_import_measure,183934.103,834658.898,3097687.964,3.237,185615.269,4.301899
perf_zip_layers_Python310_import_measure,188822.421,878432.048,3272346.674,5.990,186724.287,4.526331


In [17]:
pivot_df['total_load_time_seconds'] = pivot_df.sum(axis=1) / 1000000